In [1]:
import onnxruntime as ort
import cv2
import numpy as np
from scipy.special import softmax
import pandas as pd
from sklearn.preprocessing import LabelBinarizer

# Load training metadata and LabelBinarizer
metadata_path = r"C:\Users\General\OneDrive\Desktop\SNAD\train_metadata.csv"
metadata = pd.read_csv(metadata_path)

labels_species = metadata['binomial']
lb_species = LabelBinarizer()
lb_species.fit(np.asarray(labels_species))

# Load ONNX model
model_path = r"C:\Users\General\OneDrive\Desktop\SNAD\ViT_PreProcessing-ops11-preprocessing-int-dynam_graph.onnx"
ort_session = ort.InferenceSession(model_path)


ModuleNotFoundError: No module named 'onnxruntime'

In [ ]:
# Prediction function
def predict_image_int32(image_path, ort_session, lb_species, height=384, width=384):
    """
    Predict species from a single image using ONNX ViT that expects int32 input.
    """
    # Load image
    image = cv2.imread(image_path)
    if image is None:
        raise FileNotFoundError(f"Image not found: {image_path}")
    
    # Convert BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Resize
    image = cv2.resize(image, (width, height))
    
    # Convert to int32 (required by your model)
    image = image.astype(np.int32)
    
    # Run ONNX inference
    outputs = ort_session.run(None, {ort_session.get_inputs()[0].name: image})
    # print(outputs)
    logits = outputs[0]  # shape: (772,)
    
    # Softmax to get probabilities
    probs = softmax(logits)
    pred_class = np.argmax(probs)
    species_name = lb_species.classes_[pred_class]
    
    return pred_class, species_name

# Test single image
image_path = r"C:\Users\General\OneDrive\Desktop\SNAD\test_image.png"
pred_class, species_name = predict_image_int32(image_path, ort_session, lb_species)

print("Predicted class ID:", pred_class)
print("Predicted species name:", species_name)


Predicted class ID: 475
Predicted species name: Nerodia erythrogaster


In [ ]:
count = metadata[metadata['country'] == 'India'].shape[0]
print(count)

2904
